# Training
### Part 0 Unpacking the data

In [ ]:
import os
import zipfile

def unzip(file, destination):
    print('Unzipping to', destination)
    
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(destination)

base_dir = "/kaggle/working/"
train_images_path = os.path.join(base_dir, "images_training_rev1")
test_images_path = os.path.join(base_dir, "images_test_rev1")

# if not os.path.exists(base_dir):
unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip', base_dir)
unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip', base_dir)
unzip('/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip', base_dir)


In [ ]:
import pandas as pd

def append_ext(filename):
    """ Appends `.jpg` file extension to a filename """
    return f"{filename}.jpg"

train_sol = pd.read_csv("/kaggle/working/training_solutions_rev1.csv")
train_sol["GalaxyID"] = train_sol["GalaxyID"].apply(append_ext)
train_sol.head()

### Part 1 Preparing the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_sol,
    directory=train_images_path,
    x_col="GalaxyID",
    y_col=["Class1.1", "Class1.2", "Class1.3"],
    clases=['Early type', 'Spiral', 'Artifact'],
    subset="training",
    batch_size=32,
    shuffle=False,
    class_mode="raw",
    target_size=(224,224)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=train_sol,
    directory=train_images_path,
    x_col="GalaxyID",
    y_col=["Class1.1", "Class1.2", "Class1.3"],
    subset="validation",
    batch_size=32,
    shuffle=False,
    class_mode="raw",
    target_size=(224,224)
)

train_steps = np.ceil(train_generator.samples / train_generator.batch_size)
val_steps = np.ceil(valid_generator.samples / valid_generator.batch_size)

### Part 2: The Model

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications.vgg19 import VGG19

def build_model(num_classes):

    pre_trained_model = VGG19(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False
    )

    x = layers.Flatten()(pre_trained_model.output)

    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(1024, activation='relu')(x)

    x = layers.Dropout(0.2)(x)

    output = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(pre_trained_model.input, output)

    return model, pre_trained_model

model, pre_trained_model = build_model(3)

for layer in pre_trained_model.layers:
    layer.trainable = False


### Part 3 Compiling the Model

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-3),
    metrics=['accuracy']
)


In [ ]:
from matplotlib import pyplot as plt

def plot_history(history):
    """ 
    Retrieve a list of accuracy results on training and
    test data sets for each training epoch
    """

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.figure(dpi=100)
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.ylabel('Accuracy')
    plt.ylim([0,1])
    plt.title('Training and validation accuracy')
    plt.legend( ('training', 'validation') )
    plt.figure()

    # Plot training and validation loss per epoch
    plt.figure(dpi=100)
    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.ylabel('Loss')
    #plt.yscale('log')
    plt.title('Training and validation loss')


### Part 4 Model Training

In [ ]:
tf_history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=3,
    validation_data=valid_generator,
    validation_steps=val_steps,
    verbose=2
)

plot_history(tf_history)

In [ ]:
for layer in pre_trained_model.layers[-5:]:
    layer.trainable = True
    
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-4),
    metrics=['accuracy']
)
    
tf_history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=3,
    validation_data=valid_generator,
    validation_steps=val_steps,
    verbose=2
)

plot_history(tf_history)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = True
    
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-5),
    metrics=['accuracy']
)
    
tf_history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=3,
    validation_data=valid_generator,
    validation_steps=val_steps,
    verbose=2
)

plot_history(tf_history)


In [ ]:
model.save("/kaggle/working/my_model.h5")
del model


# Metrics

In [ ]:
from keras.models import load_model

model = load_model("/kaggle/working/my_model.h5")


In [ ]:
for layer_idx, layer in enumerate(model.layers):
    if not 'convolutional' in str(layer.__class__):
        continue
    print(layer_idx, layer.name, layer.output.shape)

visualization_model = Model(model.input, model.layers[1].output)

In [ ]:
def show(img):
    plt.figure(figsize=(8,8))
    plt.grid(False)
    plt.axis('Off')
    plt.imshow(img)
    plt.show()

next_data = valid_generator.next()
img = next_data[0][0]
show(img)

# expand dimensions so that it fakes a batch containing a single sample
img = np.expand_dims(img, axis=0)

print(f"Ground truth:\t   {next_data[1][0]}")
print(f"Model prediction: {model.predict(img)}")

In [ ]:
feature_maps = visualization_model.predict(img)

In [ ]:
square = 8
fig = plt.gcf()
fig.set_size_inches(square*2,square*2)
idx = 1
for _ in range(square):
    for _ in range(square):
        sp = plt.subplot(square, square, idx)
        sp.axis('Off')
        sp.title.set_text(str(idx-1))
        plt.imshow(feature_maps[0, :, :, idx-1])
        idx += 1

plt.show()

In [ ]:
layer_indices = [1, 2, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20]
visualization_model = Model(model.input, [model.layers[idx].output for idx in layer_indices])

In [ ]:
#@title Plot feature maps
plt.imshow(img[0])
plt.axis('Off')
plt.title('Input Image')
plt.show()

square = 8
feature_maps = visualization_model.predict(img)
for layer_idx, fmap in enumerate(feature_maps):
    fig = plt.gcf()
    fig.set_size_inches(square*2, square*2)
    fig.suptitle(model.layers[layer_indices[layer_idx]].name)
    idx = 0
    
    for _ in range(square):
        for _ in range(2):
            fm = fmap[0, :, :, idx]
            sp = plt.subplot(square, square, idx+1)
            sp.axis('Off')
            sp.title.set_text(str(idx))
            plt.imshow(fm)
            idx += 1

    plt.show()

In [ ]:
#@title Plot average feature maps
plt.imshow(img[0])
plt.axis('Off')
plt.title('Input Image')
plt.show()

fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[:5]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))
    
fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[5:10]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))
    
fig=plt.figure(figsize=(150, 150))
for layer_idx, fmap in enumerate(feature_maps[10:15]):
    sp = fig.add_subplot(1, len(feature_maps), layer_idx+1)
    sp.axis('Off')
    sp.title.set_text(model.layers[ layer_indices[layer_idx] ].name)
    plt.imshow(np.squeeze(fmap.mean(axis=-1)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(valid_generator, val_steps)
y_pred = np.argmax(Y_pred, axis=1)
y_true = np.argmax(valid_generator.labels, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

print('Classification Report')
target_names = ['Early type', 'Spiral', 'Artifact']
print(classification_report(y_true, y_pred, target_names=target_names))

# Presentation

In [ ]:
from keras.models import load_model

def show(img):
    plt.figure(figsize=(4,4))
    plt.grid(False)
    plt.axis('Off')
    plt.imshow(img)
    plt.show()

model = load_model("/kaggle/working/my_model.h5")

next_data = valid_generator.next()
img = next_data[0][0]
show(img)

# expand dimensions so that it fakes a batch containing a single sample
img = np.expand_dims(img, axis=0)

print(f"Ground truth:\t   {next_data[1][0]}")
print(f"Model prediction: {model.predict(img)}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(valid_generator, val_steps)
y_pred = np.argmax(Y_pred, axis=1)
y_true = np.argmax(valid_generator.labels, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print()

print('Classification Report')
target_names = ['Early type', 'Spiral', 'Artifact']
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))